### 選擇所在區域及城市名稱，動態顯示相關資訊及位置。
* 資料來源：
    * [城市資訊](https://restcountries.eu/)
    * [世界地圖](http://geojson.xyz/)

In [282]:
%matplotlib inline
from shapely.geometry import Point
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import requests
import json
import geopandas as gpd
from ipywidgets import interact, fixed, HTML, widgets
from IPython.display import display

In [308]:
def showMap(point):
    url = 'https://d2ad6b4ur7yvpq.cloudfront.net/naturalearth-3.3.0/ne_110m_land.geojson'
    world_layer = gpd.read_file(url)
    world_layer = world_layer.plot(figsize=(12,12),facecolor='none', edgecolor='black')
    point.plot(ax=world_layer, markersize=25, color='red')
    plt.show()    

def getCountyData(name='Algiers', df=None):
    url = 'https://www.google.com.tw/maps/place/' + name + ',12z'
    html = widgets.HTML(
        value='<a target="_blank" href='+ url +'>Go to Google Map</a>'
    )
    display(html)
    display(df.loc[df['name'] == name, ['capital', 'name', 'region', 'population', 'area', 'timezones', 'latlng']])
    
    df = df.loc[df['name'] == name, ['capital', 'name', 'region', 'population', 'area', 'timezones', 'latlng']]
    
    name = df['name'].tolist()
    capital = df['capital'].tolist()
    region = df['region'].tolist()    
    population = df['population'].tolist()
    area = df['area'].tolist()
    timezones = df['timezones'].tolist()
    latlng = df['latlng'].tolist()
    lat = [i[0] for i in latlng]
    lng = [i[1] for i in latlng]
    
    df = pd.DataFrame({
        'name': name,
        'capital': capital,
        'region': region,
        'population': population,
        'area': area,
        'timezones': timezones,
        'lat': lat,
        'lng': lng
    })

    df['Coordinates'] = list(zip(df.lng, df.lat))
    df['Coordinates'] = df['Coordinates'].apply(Point)
    county_point = gpd.GeoDataFrame(df, geometry='Coordinates')
    county_point.crs = {'init' :'epsg:4326'}
    showMap(county_point)
    
def getRegionData(region):
    url = 'https://restcountries.eu/rest/v2/region/' + region + \
    '?fields=name;capital;region;latlng;population;area;timezones'
    r = requests.get(url)
    result = r.json()
    df = pd.DataFrame(result)
    county_name = df['name'].tolist()
    interact(getCountyData, name=county_name, df=fixed(df))

interact(getRegionData, region=['Africa', 'Americas', 'Asia', 'Europe', 'Oceania'])

interactive(children=(Dropdown(description='region', options=('Africa', 'Americas', 'Asia', 'Europe', 'Oceania…

<function __main__.getRegionData(region)>